In [1]:
from tkinter import *
from tkinter.filedialog import askopenfilename
import math
import os

file=""
#anlyseButton=None
AnaliseLable=None
class FullScreenApp(object):
    def __init__(self, master, **kwargs):
        self.master=master
        master.title("Log Analizer")
        pad=3
        self._geom='200x200+0+0'
        master.geometry("{0}x{1}+0+0".format(
            master.winfo_screenwidth()-pad, master.winfo_screenheight()-pad))
        master.bind('<Escape>',self.toggle_geom)            
    def toggle_geom(self,event):
        geom=self.master.winfo_geometry()
        print(geom,self._geom)
        self.master.geometry(self._geom)
        self._geom=geom
        

In [2]:
class Menubar(object):
    def __init__(self, master):
        self.master=master
        menu = Menu(root)
        root.config(menu=menu)
        subMenu = Menu(menu)
        menu.add_cascade(label="File",menu=subMenu)
        subMenu.add_command(label="Open File",command=self.LoadFile)
        #subMenu.add_command(label="Save Output",command=doNothing)
        #subMenu.add_command(label="Print",command=printFile)
        subMenu.add_separator()
        subMenu.add_command(label="Exit",command=self.master.destroy)
        master.bind('<Escape>', self.close)
        
    def close(self, event=None):
        self.master.destroy()
        
    def LoadFile(self):
        global file
        global root
        global mainWindow
        #root.filename = askopenfilename(initialdir = "/",
        #                                title = "Select file",
        #                                filetypes = (("log files","*.log"),("all files","*.*")))
        #
        #root.filename="/home/aniket/eCommunicationsNSMObjMgr_enu_0033.log"
        root.filename="/home/aniket/eCommunicationsNSMObjMgr_enu_0033_34603239.log"
        root.filename="/home/aniket/eCommunicationsNSMObjMgr.log"
        openFile = open(root.filename, "r")
        file = openFile.readlines()
        loadFile = mainWindow.changefilename(root.filename)
        #addFileToWindow(root.filename,FileLabels)

In [3]:
class MainWindow(object):
    def __init__(self, master):
        global screenWidth
        global screenheight
        global AnaliseLable
        self.FilesFrame = Frame(root)
        self.FileWindow = Text(self.FilesFrame,width = math.floor(screenWidth / 50),
                               height= screenheight, bg="#778a98", fg = "white")
        self.FileWindow.insert(INSERT, "Files \n")
        self.FileWindow.config(state=DISABLED,font="Helvetica 15 bold")
        self.FileWindow.pack(fill=Y)
        #FileWindow.config(state=NORMAL)
        #FileWindow.insert(END, "Aniket is the greatest")
        #FileWindow.config(state=DISABLED)
        self.FilesFrame.pack(side=LEFT,fill=Y)
        
        self.TopFrame = Frame(root,bg = "#CB8B8B")
        self.AnaliseLable = Label(self.TopFrame,text = "No data to Anayse...Please select a file",
                                  font="Times 15 bold",bg = "#CB8B8B")
        self.AnaliseLable.pack(side=LEFT,padx=10,pady=10)
        self.anlyseButton = Button(self.TopFrame,text="Analise",font="Helvetica 15 bold",command=self.Analyze)
        self.anlyseButton.config(state=DISABLED)
        self.anlyseButton.pack(side=LEFT)
        self.TopFrame.pack(side=TOP,fill=X)
        
        self.WrapperFrame = Frame(root,bg="#604f4f")
        self.EvtContextFrame = ScrollableFrame(root,self.WrapperFrame,"EventContext")
        self.EvtContextFrame.forgetIt()
        self.InstanceFrame = ScrollableFrame(root,self.WrapperFrame,"Process Instances")
        self.InstanceFrame.forgetIt()
        self.StepsFrame = ScrollableFrame(root,self.WrapperFrame,"Process Steps")
        self.StepsFrame.forgetIt()
        
        self.WrapperFrame.pack(side=TOP,fill=X)
        
        self.LogsFrame = Frame(root,bg = "#383535")
        self.LogWindow = Text(self.LogsFrame, width=math.floor(screenWidth / 10),
                              height=math.floor(screenheight / 20))
        self.LogWindow.insert(INSERT, "Hello.....")        
        self.LogWindow.insert(END, "You will be able to see logs over here..")
        self.LogWindow.config(state=DISABLED)
        self.LogWindow.pack(side=RIGHT,expand=True)
        self.LogsFrame.pack(fill="both")
        self.LogsFrame.pack_forget()
    
    def changefilename(self,filename):
        self.FileWindow.config(state=NORMAL)
        if(len(self.FileWindow.get('1.0',END)) > 8):
            self.FileWindow.delete('1.0',END)
            self.FileWindow.insert(INSERT, "Files \n")
        self.FileWindow.insert(END, filename)
        self.FileWindow.config(state=DISABLED)
        self.AnaliseLable.config(text='Log file is ready to be analised')
        self.anlyseButton.config(state=NORMAL)
        
    def Analyze(self):
        anaysis = analysis(self)
        anaysis.ListEventContext(self)
        self.EvtContextFrame.showIt()

In [4]:
class ScrollableFrame(object):
    def __init__(self, master,frmeName,FrameDispay):
        global screenWidth
        self.DisplayText=FrameDispay
        self.myframe=Frame(frmeName,relief=GROOVE,bd=1,bg = "white")
        self.myframe.pack(side=LEFT,padx=2,pady=5)
        self.canvas=Canvas(self.myframe,bg = "white")
        
        self.frame=Frame(self.canvas,bg = "white")
        myscrollbar=Scrollbar(self.myframe,orient="vertical",command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=myscrollbar.set)
        myscrollbar.pack(side="right",fill="y")
        self.canvas.pack(side="left")
        self.canvas.create_window((0,0),window=self.frame,anchor='nw')
        self.frame.bind("<Configure>",self.myfunction)
        Headerlbl = Label(self.frame,text=FrameDispay,font="Helvetica 15 bold",bg = "white")
        #.grid(row=0,columnspan=2)
        Headerlbl.pack()
        #self.data()
        self.visible=False
        
    def data(self):
        for i in range(50):
            Label(self.frame,text=str(i)).grid(row=i+1,column=0)
            button = Button(self.frame,text="Hey").grid(row=i+1,column=1)

    def myfunction(self,event):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"),
                              width=math.floor(screenWidth / 4.5),height=math.floor(screenWidth / 5))
        
    def forgetIt(self):
        self.myframe.pack_forget()
        self.visible=False
                
    def showIt(self):
        self.myframe.pack(side=LEFT,padx=2,pady=5)
        self.visible=True

In [ ]:
class analysis(object):
    def __init__(self,window):
        self.window = window
        self.EventContextList = []
        if file is not None:
            window.AnaliseLable['text'] = "Analizing Data"
        else:
            window.AnaliseLable['text'] = "File has no Data"
        
    def ListEventContext(self,window):
        global file
        count = 0
        eventContextNum = 0
        if file is not None:
            for line in file:
                if "EventContext" in line:
                    EvtCon = None
                    EvtCon = EventContextCls(count,line)
                    self.EventContextList.append(EvtCon)
                    self.addButton("eventContext "+str(eventContextNum),self.window.EvtContextFrame.frame,
                              self.listProcesses,EvtCon)
                    self.addLabel(self.window.EvtContextFrame.frame,line)
                    eventContextNum += 1  
                count += 1          
                
    def addLabel(self,lWindow,lLine):
        Evtlabel = NewLabel(lWindow,lLine)
        
    def addButton(self,bName,bWindow,bCommand,bObject):
        Evtconbutton=NewButton(bWindow,bName,bCommand,bObject)
        Evtconbutton.backgroundColor="Grey"
        Evtconbutton.setBackground()
    
    def listProcesses(self,obj):
        global file
        if self.window.InstanceFrame.visible == True :
            for widget in self.window.InstanceFrame.frame.winfo_children():
                widget.destroy()
            Headerlbl = Label(self.window.InstanceFrame.frame,text=self.window.InstanceFrame.DisplayText,
                              font="Helvetica 15 bold",bg = "white")
            Headerlbl.pack()
        else:
            self.window.InstanceFrame.showIt()
        #print(obj.start,obj.end)
        processes = processInstances(self.window.InstanceFrame.frame,obj,self.window)

In [ ]:
class EventContextCls(object):
    def __init__(self,start,line):
        self.start = start
        self.end = self.findEvtContextEnd()
        self.line = line
        
    def findEvtContextEnd(self):
        global file
        lncnt=self.start
        end = None
        for line in file[self.start:]:
            if "EventContext" in line:
                end = lncnt - 1
            lncnt += 1
        return end

In [ ]:
class processInstances(object):
    def __init__(self,wondow,obj,wrapperWindow):
        self.window = wondow
        self.wrapperWindow = wrapperWindow
        self.ParentStart = obj.start
        self.ParentEnd = obj.end
        self.processStarted = False
        self.InstantiatingRec()
                
    def InstantiatingRec(self):
        global file
        lineNum = self.ParentStart
        counter = 0
        processEnd = 'N'
        for line in file[self.ParentStart:self.ParentEnd]:
            if (lineNum == processEnd):
                self.processStarted = False
            if not self.processStarted:
                if "Instantiating process definition" in line:
                    lineContent = line.split("\t")
                    pname = lineContent[len(lineContent)-1][34:-3]
                    processStart = lineNum
                    self.processStarted = True
                    processLine = line
                    processObj = process(processStart,self.ParentEnd,pname)
                    processEnd = processObj.end
                    self.addButton("Process " + str(counter), self.window, self.listSteps, processObj)
                    self.addLabel(self.window, processLine)
                    counter += 1
            lineNum += 1
    
    def addLabel(self,lWindow,lLine):
        PrcLabel = NewLabel(lWindow,lLine)
        
    def addButton(self,bName,bWindow,bCommand,bObject):
        PrcButton=NewButton(bWindow,bName,bCommand,bObject)
        if bObject.complete:
            PrcButton.backgroundColor="Grey"
        else:
            PrcButton.backgroundColor="Red"
        PrcButton.setBackground()
        
    def doNothing(self,obj):
        print("Not doing anything !!"+str(obj.start))
        print("Not doing anything !!"+str(obj.end))
        
    def listSteps(self,obj):
        global file
        if self.wrapperWindow.StepsFrame.visible == True :
            for widget in self.wrapperWindow.StepsFrame.frame.winfo_children():
                widget.destroy()
            Headerlbl = Label(self.wrapperWindow.StepsFrame.frame,text=self.wrapperWindow.StepsFrame.DisplayText,
                              font="Helvetica 15 bold",bg = "white")
            Headerlbl.pack()
        else:
            self.wrapperWindow.StepsFrame.showIt()
        #print(obj.start,obj.end)
        processes = stepInstances(self.wrapperWindow.StepsFrame.frame,obj)

In [ ]:
class process(object):
    def __init__(self,start,ParentEnd,process):
        self.start = start
        self.ParentEnd = ParentEnd
        self.process = process
        self.end,self.complete = self.findProcessEnd()
            
    def findProcessEnd(self):
        NoError = True
        lineNum = self.start
        for line in file[self.start:self.ParentEnd]:
            if "Stopping process instance of" in line:
                if str(self.process) in line:
                    if "Error" in line:
                        NoError = False
                    break
            lineNum += 1
        return lineNum,NoError

In [ ]:
class stepInstances(object):
    def __init__(self,wondow,obj):
        self.window = wondow
        self.ParentStart = obj.start
        self.ParentEnd = obj.end
        self.stepStarted = False
        self.StepRec()
                
    def StepRec(self):
        global file
        lineNum = self.ParentStart
        counter = 0
        stepEnd = 'N'
        for line in file[self.ParentStart:self.ParentEnd]:
            if (lineNum == stepEnd):
                self.stepStarted = False
            if not self.stepStarted:
                if "Instantiating step definition" in line:
                    lineContent = line.split("\t")
                    sname = lineContent[len(lineContent)-1][31:-2]
                    stepStart = lineNum
                    self.stepStarted = True
                    stepLine = line
                    stepObj = step(stepStart,self.ParentEnd,sname)
                    stepEnd = stepObj.end
                    self.addButton("Step " + str(counter), self.window, self.doNothing, stepObj)
                    self.addLabel(self.window, stepLine)
                    counter += 1
            lineNum += 1
    
    def addLabel(self,lWindow,lLine):
        PrcLabel = NewLabel(lWindow,lLine)
        
    def addButton(self,bName,bWindow,bCommand,bObject):
        PrcButton=NewButton(bWindow,bName,bCommand,bObject)
        if bObject.complete:
            PrcButton.backgroundColor="Grey"
        else:
            PrcButton.backgroundColor="Red"
        PrcButton.setBackground()
        
    def doNothing(self,obj):
        print("Not doing anything !!"+str(obj.start))
        print("Not doing anything !!"+str(obj.end))

In [ ]:
class step(object):
    def __init__(self,start,ParentEnd,step):
        self.start = start
        self.ParentEnd = ParentEnd
        self.step = step
        self.end,self.complete = self.findProcessEnd()
            
    def findProcessEnd(self):
        NoError = True
        lineNum = self.start
        for line in file[self.start:self.ParentEnd]:
            if "Stopping step instance of" in line:
                if str(self.step) in line:
                    if "Error" in line:
                        NoError = False
                    break
            lineNum += 1
        return lineNum,NoError

In [ ]:
class NewButton(object):
    def __init__(self,window,bName,bCommand,bObject):
        self.backgroundColor = "Black"
        method = lambda : bCommand(bObject)
        self.buttonName = Button(window,text=bName,command=method, fg="white",
                      highlightbackground='#3E4149',highlightthickness=2,font="Times 12 bold")
        self.buttonName.bind("<Enter>", self.flash)
        self.buttonName.bind("<Leave>", self.flashRemove)
        self.buttonName.pack()
       
    def setBackground(self):
         self.buttonName.config(bg = self.backgroundColor)
        
    def flash(self,event):
        self.buttonName.config(bg = 'lightgrey')
    
    def flashRemove(self,event):
         self.buttonName.config(bg = self.backgroundColor)

In [ ]:
class NewLabel(object):
    def __init__(self,window,line):
        self.line = line
        self.window = window
        self.Label = self.addLabel()
        
    def addLabel(self):
        lineContent = self.line.split("\t")
        tail=""
        if 'Instantiating process definition' in lineContent[5]:
            labeltext = lineContent[5][33:68]
            if len(lineContent[5][33:-3]) > 35:
                tail ="..."
        elif 'Instantiating step definition' in lineContent[5]:
            labeltext = lineContent[5][30:65]
            if len(lineContent[5][30:-3]) > 35:
                tail ="..."
        else:
            labeltext = lineContent[5][:35]
            if len(lineContent[5]) > 35:
                tail ="..."
        label = Label(self.window ,text=labeltext+tail,bg = "white")
        label.pack()

In [ ]:
from tkinter import ttk
root=Tk()

app=FullScreenApp(root)
screenWidth = root.winfo_screenwidth()
screenheight = root.winfo_screenheight()
menuBar=Menubar(root)
mainWindow=MainWindow(root)
menuBar.LoadFile()
#loadFileName=loadFileName(root,"Aniket")
root.mainloop()

Not doing anything !!12318
Not doing anything !!20952
